### 구글 논문 검색 자동화
* 제목
* 논문 링크
* 논문 요약 정보
* 저자 및 연도
* 인용 횟수 정보 

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import os

In [3]:
driver = webdriver.Chrome('chromedriver')

In [4]:
url = 'https://scholar.google.co.kr/schhp?hl=ko'
driver.get(url)

#### 논문 검색

In [5]:
search = driver.find_element_by_xpath('//*[@id="gs_hdr_tsi"]')
search_btn = driver.find_element_by_xpath('//*[@id="gs_hdr_tsb"]')

In [6]:
search.clear()   # 검색창에 입력된 글자가 있으면 지워줌
search.send_keys('machine learning')   # 검색 키워드 입력
search_btn.click()   # 검색 버튼 클릭

#### 제목 가져오기

In [7]:
page = driver.page_source
soup = BeautifulSoup(page,'lxml')
all_dat = soup.find('div',class_='gs_ri').find('a')
all_dat

<a data-clk="hl=ko&amp;sa=T&amp;ct=res&amp;cd=0&amp;d=6164665760994927917&amp;ei=2ybPXq_2M9W4ygSTspGIAQ" data-clk-atid="LaUdC9lKjVUJ" href="https://books.google.co.kr/books?hl=ko&amp;lr=&amp;id=tZnSDwAAQBAJ&amp;oi=fnd&amp;pg=PR7&amp;dq=machine+learning&amp;ots=F2_PbW4tvf&amp;sig=o58vBjUcGudKBfDOVef4ud7icog" id="LaUdC9lKjVUJ">Introduction to <b>machine learning</b></a>

In [8]:
all_dat.text

'Introduction to machine learning'

In [14]:
all_dat = soup.find_all('div',class_='gs_ri')[1].find('a')
all_dat.text   # 정보를 제대로 가져오지 못함

'KP Murphy'

In [22]:
all_dat = soup.find('div',id='gs_res_ccl_mid').find_all('div',class_='gs_r gs_or gs_scl')
all_dat[5].find('div',class_='gs_ri').find('a').text

'Genetic algorithms and machine learning'

In [23]:
keyword = 'gan'

In [24]:
import time 

title = []
des = []
title_url = []
ref_cnt = []
author_year = []

time_lay = [2,3,4,5,6,7]
cnt = 0
for i in range(0, 100, 10):
    cnt =  cnt + 1
    url1 = 'https://scholar.google.co.kr/scholar?start='+ str(i) + '&q=' + keyword + '&hl=ko&as_sdt=0,5'
    driver.get(url1)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    print( soup.title )
    
    all_dat = soup.find('div', id='gs_res_ccl_mid').find_all("div", class_='gs_r gs_or gs_scl')
    for item in all_dat:
        # 제목
        temp1 = item.find("div", class_='gs_ri').find('a')
        if temp1 is not None:
            tmp_txt = temp1.text
            title.append(tmp_txt)
        else:
            title.append("")
          
        # 요약 정보
        t2 = item.find('div', class_="gs_rs")
        if t2 is not None:
            temp2 = item.find('div', class_="gs_rs").text
            des.append(temp2)
        else:
            des.append("")
            
        # 논문 링크 가져오기
        t3 = item.find("div", class_='gs_ri').find('a')
        if t3 is not None:
            temp3 = item.find("div", class_='gs_ri').find('a')['href']
            title_url.append(temp3)
        else:
            title_url.append("")
            
        # 저자 및 연도
        t4 = item.find("div", class_='gs_a')
        if t4 is not None:
            tmp_txt = t4.text
            author_year.append(tmp_txt)
        else:
            author_year.append("")

    # 인용횟수 정보 가져오기
    # //*[@id="gs_res_ccl_mid"]/div[2]/div[2]/div[3]/a[3]
    des_all = driver.find_elements_by_partial_link_text("인용")
    for item in des_all:
        # 링크
        temp1 = item.text
        # print(temp1)
        ref_cnt.append(temp1)

    print(len(title), len(des), len(title_url), len(ref_cnt), len(author_year) )
    time.sleep(time_lay[cnt%6])
    
print("정보 수집 완료")

<title>gan - Google 학술 검색</title>
10 10 10 10 10
<title>gan - Google 학술 검색</title>
20 20 20 20 20
<title>gan - Google 학술 검색</title>
30 30 30 30 30
<title>gan - Google 학술 검색</title>
40 40 40 40 40
<title>gan - Google 학술 검색</title>
50 50 50 50 50
<title>gan - Google 학술 검색</title>
60 60 60 60 60
<title>gan - Google 학술 검색</title>
70 70 70 70 70
<title>gan - Google 학술 검색</title>
80 80 80 80 80
<title>gan - Google 학술 검색</title>
90 90 90 90 90
<title>gan - Google 학술 검색</title>
100 100 100 100 100
정보 수집 완료


In [25]:
import pandas as pd

dat = { "title":title, "description":des, "인용횟수":ref_cnt, "논문링크":title_url, "저자및년도":author_year }
excel_dat = pd.DataFrame(dat)

excel_dat.to_excel("data.xlsx", index=False)